In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # to read records completely
import pickle
import numpy as np
import itertools
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

## Previous data prep necessary for c) and d)

In [2]:
df = pd.read_pickle('data_clean.pkl')

In [5]:
models = pd.read_csv('models.csv', header=None, names = ['brand', 'model'])
models['brand'] = models['brand'].str.replace(r'[^\w\s]+', '', regex=True) # clean some punctuation errors (nissan.)
models = models[~models.brand.isin(['car', 'sedan', 'problem'])] # remove rows that are not brands

In [6]:
df.Message.str.contains('said').sum()

370

In [7]:
df.Message_words = df.Message_words.apply(lambda x: [i.lower() for i in x])

### Replace models with brands

In [8]:
messages = df.Message_words.to_list()

In [9]:
# numpy.where to do the replacement. Replaces only first appearance of brand.
# ~ 5-10 minutes of execute
messages2 = []
t = 0
for m in messages:
    for i,row in models.iterrows():
        m = np.where(m == row['model'], row['brand'], m)
        
    t+=1
    
    #print(t)
        
    messages2.append(m)
        
df['Message_words_v2'] = messages2

In [10]:
# let's find the occurences of changes
m1 = df.Message_words.to_list()
m2 = df.Message_words_v2.to_list()

c = 0
W = 0
for i in range(len(m2)):
    for j in range(len(m2[i])):
        if m1[i][j] != m2[i][j]: #word by word comparison
            c +=1
        W += 1 # if we want to count all words

In [11]:
print(f'{c} changes of models')

6368 changes of models


In [13]:
df[['Message_words', 'Message_words_v2']].head(10)

Message_words  \
0                                                                                                                                                                                                                                                                             [need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]   
1                                                                                                                                                                                                                                                                                [im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]   
2                                                                                                                                                                                           [buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]   
3  [looking, 1, 4, choices, replace, 99gs400, ive, read, everything, forums, seen, much, conflicting, info4, optionslexus, ls430, customluxury, packageinfiniti, q45, premium, pckgaudi, a6, 42, loaded, audi, a8, stripped, loadedchicago, wintershad, blizzaks, gsworked, fine, 95q, pig, snow, im, sure, 2002q, world, away, ive, heard, audi, qc, problems, like, maintenance, included, im, happy, lexus, driver, q, appears, offer, lease, economy, would, think, could, command, attractive, package, go, wrong, 4, missing, something]   
4                                                                                                                                                                                                                                                    [generally, 34, year, old, car, range, good, one, basically, sound, possible, get, one, still, warrenty, make, sure, trusted, maxima, mechanic, goes, thoroughly, v6, one, best, rated, 10, years, personally, amazed, great, strides, american, quality, recently, olds, may, bad, idea]   
5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      [maxima, intrigue, excellent, choices, midsize, sedans]   
6                                                                                                                                                                                                                                                                                                                                                                                                                                                     [kind, budget, would, go, a6, 42, best, combination, sport, luxury, amongst, 4, choices]   
7                                                                                                                                                                                                                                          [actually, agree, a6, looked, specs, e

In [15]:
#Now we have cases where the text says "nissan", "nissan" because we replaced the model. Let's clean that
#Removing duplicates in lists while keeping order
#import itertools
def remove_consecutive_duplicate (text):
    return np.array([i for i, j in itertools.groupby(text)])

df["Message_words_v3"] = df["Message_words_v2"].apply(remove_consecutive_duplicate)

## Link attributes to models

In [19]:
# first lets count brand mentions in each record

In [67]:

#Get a unique list of brands
brands_list = models.brand.drop_duplicates().to_list()

brands_list.extend(['lexus', 'ferrari', 'merzedesbenz', 'tesla','gm', 'peugeot', 'jeep', 'bentley', 'fiat'])

In [68]:
messages = df.Message_words_v3.to_list()


brands_in_message = []
for m in messages:
    
    brands_mentioned = []
    for brand in brands_list:
        if len(np.where(m == brand)[0]) >0:
            brands_mentioned.append(brand)
            
    brands_in_message.append(brands_mentioned)
        
    

<ipython-input-68-727eeecb9ebd>:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if len(np.where(m == brand)[0]) >0:


In [69]:
df['Brands_in_message'] = brands_in_message
df['Qty_brands'] = df.Brands_in_message.apply(lambda x: len(x))
df['Qty_brands'].value_counts()

# 1291 messages don't mention any brand (remove them from analysis?)
# Most messages discuss a single brand
# Also common to compare 2 brands

0     1244
1     1175
2      774
3      464
4      235
5      120
6       74
7       31
8       17
10       7
11       3
12       3
9        2
16       1
13       1
Name: Qty_brands, dtype: int64

In [70]:
df.loc[df.Qty_brands == 0, 'Message_words_v3'].head(10) # to review records with no brand mentions
# most of them are for seeking advice

0                                                                                       [need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]
2     [buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]
8                                                                                                                                                                                                                                                                                      [id, pass, intrigue, one, reaso

In [71]:
# First approach: link all words found between brand mention and next brand mention.

# Except for first brand which also receives words since beginning of message

m = df.Message_words_v3[9] #record 9 is a good exaple

def get_attr_in_the_right(m, brand_list):

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]


    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

    brand_list = list(brand_dic.values()) + [len(m)]

    review={}
    for i, tup in enumerate(brand_dic):
#         print(brand_dic[tup])
#         print(tup)
        
        if i == 0:
            
            review[tup] = m[0:brand_list[1]]
        else:
            review[tup] = m[brand_list[i]:brand_list[i+1]]
            
    return review



get_attr_in_the_right(m, brand_list)        

{'honda': array(['would', 'recommend', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
        '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [72]:
# now we run it for all records

brand_reviews_1st_approach = []

for m in df.Message_words_v3:

    brand_reviews_1st_approach.append(get_attr_in_the_right(m, brand_list))

# generates a list (1 entry per row) of dictionaries {brand: part of text corresponding}

<ipython-input-71-3c3f4450bffc>:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ix = np.where(m == brand)[0]


In [76]:
# 2nd approach: 5 words to each side 

# First approach: link all words found between brand mention and next brand mention.

# Except for first brand which also receives words since beginning of message

m = df.Message_words_v3[9] #record 9 is a good exaple

def get_attr_n_per_side(m, n, brand_list):

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]


    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))
    
    #print(len(brand_dic))

    brand_list = list(brand_dic.values()) + [len(m)]
    
    #print(brand_list)

    review={}
    for i, tup in enumerate(brand_dic):
#         print(brand_dic[tup])
#         print(tup)

        left_loc = (brand_list[i] - n)
        left_loc = max(0, left_loc) # to avoid out of range indexing
        
        right_loc = (brand_list[i] + n)
        right_loc = min(len(m), right_loc) # to avoid OOF indexing
        
        review[tup] = m[left_loc:right_loc]
            
    return review



get_attr_n_per_side(m, 5, brand_list)        

{'honda': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new'],
       dtype='<U12'),
 'toyota': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new',
        'nissan'], dtype='<U12'),
 'nissan': array(['honda', 'toyota', 'choices', 'sedan', 'new', 'nissan', 'looks',
        'impressive', 'would', 'wait'], dtype='<U12'),
 'bmw': array(['wait', 'reliability', '2002', 'proventhe', 'new', 'bmw', '525',
        'another', 'nice', 'car'], dtype='<U12')}

In [77]:
m = df.Message_words_v3[9] 

m

array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new',
       'nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
       '2002', 'proventhe', 'new', 'bmw', '525', 'another', 'nice', 'car',
       'rated', 'highly', 'many', 'auto', 'publications'], dtype='<U12')

In [78]:
get_attr_n_per_side(m, 4, brand_list)

{'honda': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan'],
       dtype='<U12'),
 'toyota': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new'],
       dtype='<U12'),
 'nissan': array(['toyota', 'choices', 'sedan', 'new', 'nissan', 'looks',
        'impressive', 'would'], dtype='<U12'),
 'bmw': array(['reliability', '2002', 'proventhe', 'new', 'bmw', '525', 'another',
        'nice'], dtype='<U12')}

In [79]:
get_attr_in_the_right(m, brand_list)        

{'honda': array(['would', 'recommend', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
        '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [80]:
# now we run it for all records

brand_reviews_2nd_approach = []

for m in df.Message_words_v3:

    brand_reviews_2nd_approach.append(get_attr_n_per_side(m,4, brand_list))

# generates a list (1 entry per row) of dictionaries {brand: part of text corresponding}

<ipython-input-76-ef5b6bba9520>:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ix = np.where(m == brand)[0]


In [81]:
len(brand_reviews_2nd_approach) # each record becomes a single dictionary with all its brand mentions and linked words

4151

In [82]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
review_accum_1st = {i:[] for i in brands_list}

for brand in brands_list:
    for review in brand_reviews_1st_approach:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum_1st[brand].extend(review[single_review])

In [85]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
review_accum_2nd = {i:[] for i in brands_list}

for brand in brands_list:
    for review in brand_reviews_2nd_approach:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum_2nd[brand].extend(review[single_review])

In [84]:
review_accum_1st

{'acura': ['acura',
  'come',
  'v8',
  'model',
  'acura',
  'rl',
  'might',
  'try',
  'acura',
  'tl',
  'smallest',
  'caddy',
  'rwd',
  'ctsid',
  'pick',
  'acura',
  'tl',
  'type',
  'best',
  'value',
  'edmunds',
  'editors',
  'choice',
  'well',
  'dont',
  'remember',
  'exactly',
  'going',
  'know',
  'got',
  'ton',
  'bad',
  'press',
  'sully',
  'name',
  'bit',
  'kind',
  'like',
  'isuzu',
  'trooper',
  'flipping',
  'killed',
  'acura',
  'version',
  'remember',
  'current',
  'grand',
  'cherokee',
  'sudden',
  'acceleration',
  'im',
  'looking',
  'buying',
  'new',
  'family',
  'sedan',
  'near',
  'luxury',
  'class',
  'looking',
  '1',
  'acura',
  'tl',
  '32',
  'type',
  '2',
  '02',
  'acura',
  'acura',
  'great',
  'acura',
  'well',
  'sporty',
  'nissan',
  'upgrade',
  'honda',
  'think',
  'claywater',
  'godeacs',
  'dead',
  'cars',
  'near',
  'luxury',
  'arena',
  'great',
  'bang',
  'buck',
  'cars',
  'nice',
  'engines',
  'interio

In [86]:
review_accum_2nd

{'acura': ['leftover',
  'available',
  'scramble',
  'thinking',
  'acura',
  'come',
  'v8',
  'model',
  'range',
  'new',
  'toyota',
  'told',
  'acura',
  'rl',
  'infiniti',
  'j30',
  'might',
  'try',
  'acura',
  'tl',
  'lexus',
  'es300',
  'caddy',
  'rwd',
  'ctsid',
  'pick',
  'acura',
  'tl',
  'type',
  'best',
  'isuzu',
  'trooper',
  'flipping',
  'killed',
  'acura',
  'version',
  'remember',
  'current',
  'luxury',
  'class',
  'looking',
  '1',
  'acura',
  'tl',
  '32',
  'type',
  'would',
  'go',
  'toyota',
  'xle',
  'acura',
  'nissan',
  'look',
  'sharper',
  'concerned',
  'primarily',
  'resale',
  'honda',
  'acura',
  'great',
  'toyota',
  'bad',
  'may',
  'want',
  'take',
  'look',
  'acura',
  'well',
  'sporty',
  'nissan',
  'interiors',
  'wont',
  'kill',
  'wallet',
  'acura',
  'may',
  'boring',
  'looking',
  'rough',
  'time',
  'deciding',
  'acura',
  'tls',
  'audi',
  '18t',
  'like',
  'acura',
  'higher',
  'reliability',
  'plu

In [32]:
## from here, we are ready to start counting attributes per brand
## we also need to consider replacing some similar attributes before doing the final count 
## I will do one example with Audi but it could become a function and run in a loop to all the brands 

In [33]:
# first lets discover popular attributes regardless of brands

all_attributes = []

for i in list(review_accum.values()):
    all_attributes.extend(i)
    
words = pd.Series(all_attributes)

words.value_counts().head(40) # this shows the frequencies of each word
# the most "rustic" approach would be to literally print them in order and take note of the ones that can be considered "attributes"

car           3152
honda         1746
like          1634
would         1600
get           1453
new           1394
toyota        1341
cars          1324
good          1249
one           1178
also           964
nissan         963
im             943
dont           883
drive          880
miles          823
much           808
want           802
better         796
years          776
used           766
think          750
volkswagen     738
price          737
looking        714
well           699
really         669
buy            583
know           579
go             560
even           560
chevrolet      556
seat           551
need           550
great          533
year           532
could          529
ive            495
driving        487
might          481
dtype: int64

In [34]:
words[~words.isin(brands_list)].value_counts().head(60)


car            3152
like           1634
would          1600
get            1453
new            1394
cars           1324
good           1249
one            1178
also            964
im              943
dont            883
drive           880
miles           823
much            808
want            802
better          796
years           776
used            766
think           750
price           737
looking         714
well            699
really          669
buy             583
know            579
even            560
go              560
need            550
great           533
year            532
could           529
ive             495
driving         487
might           481
time            479
look            466
lot             456
best            454
engine          454
something       446
3               445
reliability     442
id              441
nice            439
going           434
vehicle         433
civic           431
old             424
probably        419
still           409


In [80]:
words_count = words.value_counts()
words_nobrands_count = words[~words.isin(brands_list)].value_counts()

### Stemming

In [36]:
#Stemming
stemmer = PorterStemmer()

In [37]:
def stem_list (text):
    return pd.Series([stemmer.stem(w) for w in text])

In [38]:
stemmed_words = stem_list(words)
len(stemmed_words)

191416

In [39]:
stemmed_words.value_counts().head(60)

car           4477
like          1924
get           1854
honda         1746
would         1600
look          1585
drive         1477
new           1394
toyota        1341
year          1308
one           1270
good          1251
use           1023
go             994
want           993
also           964
nissan         963
im             943
price          925
mile           900
think          890
dont           883
buy            848
much           808
reliabl        806
better         799
volkswagen     741
need           738
well           699
vehicl         690
realli         669
know           635
model          633
time           589
seat           581
make           581
lot            578
even           562
chevrolet      558
engin          557
great          533
could          529
dealer         506
ive            495
consid         485
might          481
sedan          463
nice           459
civic          459
best           455
someth         446
3              445
audi        

In [40]:
stemmed_words[~stemmed_words.isin(brands_list)].value_counts().head(60)


car        4477
like       1924
get        1854
would      1600
look       1585
drive      1477
new        1394
year       1308
one        1270
good       1251
use        1023
go          994
want        993
also        964
im          943
price       925
mile        900
think       890
dont        883
buy         848
much        808
reliabl     806
better      799
need        738
well        699
vehicl      690
realli      669
know        635
model       633
time        589
make        581
lot         578
even        562
engin       557
great       533
could       529
dealer      506
ive         495
consid      485
might       481
sedan       463
nice        459
civic       459
best        455
someth      446
3           445
id          442
leas        442
old         441
find        422
option      422
probabl     420
test        419
seem        415
back        409
4           409
still       409
cost        403
less        402
around      396
dtype: int64

In [41]:
stemwords_count = stemmed_words.value_counts()
stemwords_nobrands_count = stemmed_words[~stemmed_words.isin(brands_list)].value_counts()

In [77]:
type(stemwords_count)

pandas.core.series.Series

In [42]:
#Export value count series to a single multi-sheet Excel
#pip install xlsxwriter
writer = pd.ExcelWriter('attribute_count.xlsx', engine='xlsxwriter')


In [43]:
words_count.to_excel(writer, sheet_name='words_valuecounts')
words_nobrands_count.to_excel(writer, sheet_name='words_nobrands_valuecounts')
stemwords_count.to_excel(writer, sheet_name='stemwords_valuecounts')
stemwords_nobrands_count.to_excel(writer, sheet_name='stemwords_nobrands_valuecounts')

In [44]:
writer.save()


In [82]:
#For words_nobrands_count remove words that appear 2x or less
words_nobrands_count = words_nobrands_count[words_nobrands_count > 2]
words_nobrands_count

car         3152
like        1634
would       1600
get         1453
new         1394
            ... 
nope           3
scrape         3
850            3
modelsi        3
rearview       3
Length: 5361, dtype: int64

In [83]:
words_nobrands_count.to_csv("attribute_count_final.csv")

### TF-IDF

In [53]:
df

Date     User_Id  \
0    2001-09-01         pat   
1    2001-09-01    willow25   
2    2001-10-01      dindak   
3    2001-10-01     peteri1   
4    2001-10-01  mrdetailer   
...         ...         ...   
4146 2021-02-01       kyfdx   
4147 2021-02-01       backy   
4148 2021-02-01   RayeEliza   
4149 2021-02-01     mlevine   
4150 2021-02-01     mcdawgg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Message  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Need help choosing your next vehicle? Tell us your price range, are you thinking New or Used, buying or leasing, what features are must-haves, what other thoughts are on your mind, and let us give you a hand! Tell us your criteria as specifically as you can.You'll find lots of helpful folks here who can give you useful suggestions.    
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          I'm buying a "new" car by the end of the year and my first choice is a Nissan Maxima. I've read that they are very reliable and hold their value well. My budget is probably going to allow for a 98-99 and I was wondering if it's worth buying a car that's almost 3-4 years old. Any suggestions or comments would help a lot.willow25    
2                                                                                                                                                                                                          

In [61]:
def listtostring(text):
    return " ".join(text) 

In [63]:
df["Message_words_v4"] = df["Message_words_v3"].apply(listtostring)

In [66]:
vectorizer = TfidfVectorizer()

In [69]:
X = vectorizer.fit_transform(df["Message_words_v4"].tolist())

In [71]:
df_tfidf = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"])
df_tfidf = df_tfidf.sort_values('TF-IDF', ascending=False)

In [73]:
df_tfidf.to_csv("attribute_tfidf.csv")

In [45]:
# once we have the list of the 5 most popular attributes, my guess is we have to calculate the uplift scores of said attribute
# against the 5 top brands found in letter A). Then the uplift should tell us which brand is more related to which attribute

In [46]:
# let's say "reliability" is one of the popular attributes 
# and the top 5 brands are: honda, nissan, audi, bwm, toyota
# let's get the inputs necessary to calculate uplift of reliability in nissan

In [47]:
n=0
for i in  df.Brands_in_message:
    if 'nissan' in i:
        n += 1
n
#nissan appears in 525 rows 

525

In [48]:
df_nissan = pd.DataFrame(review_accum['nissan'], columns = ['word'])
(df_nissan.word == 'reliability').sum() # 73 mentions of "relibaility" in honda

# and 28 times the term "reliability" was mentioned alongside a nissan

28

In [49]:
# how about honda?

n=0
for i in  df.Brands_in_message:
    if 'honda' in i:
        n += 1
print(n)


df_honda = pd.DataFrame(review_accum['honda'], columns = ['word'])
(df_honda.word == 'reliability').sum() # 73 mentions of "relibaility" in honda


924


73

In [50]:
73/924 # not the exact uplift formula but a quick validation

0.07900432900432901

In [51]:
28/525 # vs nissan

0.05333333333333334

In [52]:
# honda has more prevalence as a reliable brand than nissan (pending to adjust for uplift real formula)

to consider:

right now the code counts all the appearances of an attribute, even if they come from the same record; I think that is not the exact definition of the uplift formula. I don't think it will change the numbers very much, but would be good to refine it if we have the time